# Alle benötigten Python-Pakete installieren

In [1]:
!pip install langchain langchain_community
!pip install pypdf
!pip install langchain_openai
!pip install chromadb

# Dokumente zusammenstellen

In [2]:
URL = "https://www.uni-hannover.de/fileadmin/luh/studium/ordnungen/stpo/f_inform_bpo24.pdf"
import urllib.request
urllib.request.urlretrieve(URL, "dokument1.pdf")

('dokument1.pdf', <http.client.HTTPMessage at 0x7619e0580980>)

In [3]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("dokument1.pdf")
pages = loader.load()

In [4]:
len(pages)

34

In [5]:
type(pages)

list

In [6]:
pages[0].page_content

'Verkündungsblatt der Gottfried Wilhelm Leibniz Universität Hannover vom 24.09.2024 13/2024 \nSeite 56 \nDer Dekan der Fakultät für Elektrotechnik und Informatik der Gottfried Wilhelm Leibniz Universität Hannover hat am \n17.07.2024 per Eilbeschluss die nachstehende Änderung der Prüfungsordnung für den Bachelorstudiengang Informatik \nvom 23.03.2017, in der Fassung der letzten Änderung beschlossen. Das Präsidium hat die Änderung am 28.08.2024 \ngemäß § 37 Abs. 1 Nr. 5. b) des Niedersächsischen Hochschulgesetzes genehmigt. Sie tritt nach ihrer hochschulöffentli-\nchen Bekanntmachung im Verkündungsblatt der Gottfried Wilhelm Leibniz Universität Hannover zum 01.10.2024 in \nKraft. \n \nÄnderung der Prüfungsordnung für den Bachelorstudiengang Informatik \nan der Gottfried Wilhelm Leibniz Universität Hannover vom 23.03.2017,  \nmit Änderungen vom 18.09.2018, 13.08.2019, 01.09.2020, 30.07.2021, 08.09.2022 (berichtigt am \n07.11.2022) und 24.04.2024 (berichtigt am 22.05.2024) \n \nDie\n Fakul

In [7]:
type( pages[0].page_content )

str

# Dokumente in Stücke zerschneiden

In [9]:
text_len = len(pages[0].page_content)
text_len

2374

In [10]:
pages[0].page_content[text_len//2:]

'\nnung erlassen: \nÜbersicht \n \nErster Teil: Allgemeines  \n§ 1 Zweck der Prüfung und Hochschulgrad \n§ 2 Dauer und Gliederung des Studiums, Zwischenprüfung \n§ 3 Zuständigkeit (Studiendekanin oder  Studiendekan, Prüfungsausschuss) \n \nZweiter Teil: Bachelorprüfung \n§ 4 Aufbau und Inhalt der Prüfung \n§ 5 Prüferinnen und Prüfer sowie Beisitzende \n§ 6 Studien- und Prüfungsleistungen \n§ 6a  Digitale Prüfungsformate \n§ 7 Bachelorarbeit  \n§ 8 Bestehen und Nichtbestehen  \n§ 9 Zusätzliche Module und Prüfungen \n§ 10 Anerkennung bereits erbrachter Studien- und Prüfungsleistungen \n§ 10a Einstufungsprüfung für Geflüchtete \n§ 11 Erbringung von Prüfungsleistungen durch Juniorstudierende  \n \nDritter Teil: Prüfungsverfahren \n§ 12 Zulassung zu Prüfungsleistungen \n§ 13 Anmeldung \n§ 14 Wiederholung \n§ 15 Versäumnis, Rückt ritt, Fristverlängerung \n§ 16 Prüfungsmodalitäten in Härtefällen \n§ 17 Bewertung von Prüfungsleistungen \n§ 18 Täuschung, Ordnungsverstoß \n§ 19 Leistungspunkte u

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents( pages )

In [12]:
type(chunks)

list

In [13]:
len(chunks)

226

In [14]:
chunks[0].page_content

'Verkündungsblatt der Gottfried Wilhelm Leibniz Universität Hannover vom 24.09.2024 13/2024 \nSeite 56 \nDer Dekan der Fakultät für Elektrotechnik und Informatik der Gottfried Wilhelm Leibniz Universität Hannover hat am \n17.07.2024 per Eilbeschluss die nachstehende Änderung der Prüfungsordnung für den Bachelorstudiengang Informatik \nvom 23.03.2017, in der Fassung der letzten Änderung beschlossen. Das Präsidium hat die Änderung am 28.08.2024'

# Vektordatenbank erstellen

In [ ]:
# 1. Dokument(e) bereitstellen
URL = "https://www.uni-hannover.de/fileadmin/luh/studium/ordnungen/stpo/f_inform_bpo24.pdf"
import urllib.request
urllib.request.urlretrieve(URL, "dokument1.pdf")

# 2. Dokument(e) einlesen
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("dokument1.pdf")
pages = loader.load()

# 3. Dokument(e) zerstückeln in "Chunks"
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents( pages )

# 4. ChomaDB erstellen
api_key = "<insert your OpenAI API key here>"
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
embeddings = OpenAIEmbeddings(api_key=api_key)
vectorstore = Chroma.from_documents(
    chunks,
    embedding=embeddings,
    persist_directory="chromadb_studienordnungen"
)

# Beispielabfrage (Retrieval)

In [17]:
prompt = "Wieviele Personen sind in der Prüfungskommission?"

docs = vectorstore.similarity_search(prompt, k=5)

In [18]:
type(docs)

list

In [19]:
len(docs)

5

In [20]:
docs[0]

Document(metadata={'source': 'dokument1.pdf', 'page': 1, 'page_label': '2', 'keywords': 'Änderung der Prüfungsordnung für den Bachelorstudiengang Informatik an der Gottfried Wilhelm Leibniz Universität Hannover vom 23.03.2017, mit Änderungen vom 18.09.2018, 13.08.2019, 01.09.2020, 30.07.2021, 08.09.2022 (berichtigt am 07.11.2022) und 24.04.2024 (berichtigt am 22.05.2024) Verkündungsblatt Nr. 13/2024', 'total_pages': 34, 'subject': 'Änderung der Prüfungsordnung für den Bachelorstudiengang Informatik an der Gottfried Wilhelm Leibniz Universität Hannover vom 23.03.2017, mit Änderungen vom 18.09.2018, 13.08.2019, 01.09.2020, 30.07.2021, 08.09.2022 (berichtigt am 07.11.2022) und 24.04.2024 (berichtigt am 22.05.2024) Verkündungsblatt Nr. 13/2024', 'title': 'Änderung der Prüfungsordnung für den Bachelorstudiengang Informatik an der Gottfried Wilhelm Leibniz Universität Hannover vom 23.03.2017, mit Änderungen vom 18.09.2018, 13.08.2019, 01.09.2020, 30.07.2021, 08.09.2022 (berichtigt am 07.11.2

# Augmentierten Prompt erstellen

In [21]:
prompt

'Wieviele Personen sind in der Prüfungskommission?'

In [22]:
docs[0].page_content

'gung dieser Aufgaben eingesetzt wird. \n(2) \n1Dem Prüfungsausschuss gehören fünf Mitglieder an, und zwar drei Mitglieder aus der Hochschullehrer-\ngruppe, ein Mitglied, das die Mitarbeitergruppe vertritt und in der Lehre tätig ist, sowie ein Mitglied der \nStudierendengruppe. 2Die Mitglieder des Prüfungsausschusses sowie deren Vertretungen werden durch \ndie jeweiligen Gruppenvertretungen benannt. 3Der Vorsitz und der stellvertretende Vorsitz, über die der'

In [25]:
kontext_infos = ""
for doc in docs:
    kontext_infos += "\n----\n" + doc.page_content + "\n"

prompt_aug = f"""Beantworte die folgende Frage nur auf Basis der im Kontext 
von RAG extrahierten Chunks:
----
{prompt}
----
Chunks:
{kontext_infos}
"""

print(prompt_aug)

Beantworte die folgende Frage nur auf Basis der im Kontext 
von RAG extrahierten Chunks:
----
Wieviele Personen sind in der Prüfungskommission?
----
Chunks:

----
gung dieser Aufgaben eingesetzt wird. 
(2) 
1Dem Prüfungsausschuss gehören fünf Mitglieder an, und zwar drei Mitglieder aus der Hochschullehrer-
gruppe, ein Mitglied, das die Mitarbeitergruppe vertritt und in der Lehre tätig ist, sowie ein Mitglied der 
Studierendengruppe. 2Die Mitglieder des Prüfungsausschusses sowie deren Vertretungen werden durch 
die jeweiligen Gruppenvertretungen benannt. 3Der Vorsitz und der stellvertretende Vorsitz, über die der

----
Prüfungsausschuss entscheidet, müssen von Mitgliedern der Hochschullehrergruppe oder habilitierten 
Mitgliedern ausgeübt werden. 4Das studentische Mitglied hat in Bewertungsfragen und bei der Entschei-
dung über die Anerkennung von Prüfungsleistungen nur beratende Stimme. 5Die Studiendekanin oder 
der Studiendekan kann, falls sie oder er nicht als Mitglied des Prüfungsaus

# RAG in einem Skript (wenn Vector Datenbank schon da)

In [ ]:
# 1. Was will der Anfragende wissen?
prompt = "Wieviele Personen sind in der Prüfungskommission?"

# 2. Vektordatenbank wiederherstellen
api_key = "<insert your OpenAI API key here>"
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
embeddings = OpenAIEmbeddings(api_key=api_key)
vectorstore = Chroma(embedding_function=embeddings,
                     persist_directory="chromadb_studienordnungen"
                    )

# 3. Chunk Retrieval
docs = vectorstore.similarity_search(prompt, k=5)

# 4. Erstelle augmentierten Prompt
kontext_infos = ""
for doc in docs:
    kontext_infos += "\n----\n" + doc.page_content + "\n"
prompt_aug = f"""Beantworte die folgende Frage NUR auf Basis der im Kontext 
von RAG extrahierten Chunks:
----
{prompt}
----
Chunks:
{kontext_infos}
"""
print(prompt_aug)

# 5. Augmentierten Prompt an ein LLM stellen
from openai import OpenAI
client = OpenAI(api_key=api_key)
response = client.responses.create(
    model="gpt-5.2",
    input=prompt_aug,
    temperature=0.0
)
print("\n\n")
print("ANTWORT DES MODELLS:")
print("-" * 50)
print(response.output_text)

/tmp/ipykernel_20530/2346370852.py:9: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(embedding_function=embeddings,


Beantworte die folgende Frage NUR auf Basis der im Kontext 
von RAG extrahierten Chunks:
----
Wieviele Personen sind in der Prüfungskommission?
----
Chunks:

----
gung dieser Aufgaben eingesetzt wird. 
(2) 
1Dem Prüfungsausschuss gehören fünf Mitglieder an, und zwar drei Mitglieder aus der Hochschullehrer-
gruppe, ein Mitglied, das die Mitarbeitergruppe vertritt und in der Lehre tätig ist, sowie ein Mitglied der 
Studierendengruppe. 2Die Mitglieder des Prüfungsausschusses sowie deren Vertretungen werden durch 
die jeweiligen Gruppenvertretungen benannt. 3Der Vorsitz und der stellvertretende Vorsitz, über die der

----
Prüfungsausschuss entscheidet, müssen von Mitgliedern der Hochschullehrergruppe oder habilitierten 
Mitgliedern ausgeübt werden. 4Das studentische Mitglied hat in Bewertungsfragen und bei der Entschei-
dung über die Anerkennung von Prüfungsleistungen nur beratende Stimme. 5Die Studiendekanin oder 
der Studiendekan kann, falls sie oder er nicht als Mitglied des Prüfungsaus